# Knowledge Graph Querying: Competency Questions (CQs)
This notebook shows how to query KGs to answer the CQs related to the OntoBOT ontology.

We will:
- load local RDF graphs
- Formulate SPARQL queries corresponding to CQ

#### 🔧 Setup 🔧

In [90]:
from rdflib import Graph, Namespace
from collections import defaultdict
from rdflib.namespace import RDFS, FOAF, DCTERMS

import pandas as pd

In [91]:
# Load graph
g = Graph()
g.parse("./scenarios_kgs.ttl", format="ttl")
g.parse("./case_study/hi-thesaurus.ttl", format="ttl")
g.parse("../hi-ontology/hi-ontology.ttl", format="ttl")

<Graph identifier=N1436c576d21540bc9d4eb996cae3c83b (<class 'rdflib.graph.Graph'>)>

In [92]:
HI = Namespace("https://w3id.org/hi-ontology#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
HINT = Namespace("http://w3id.org/hi-thesaurus/")
HI_S1 = Namespace("https://w3id.org/hi-ontology/scenarios/1")
HI_S2 = Namespace("https://w3id.org/hi-ontology/scenarios/2")
HI_S3 = Namespace("https://w3id.org/hi-ontology/scenarios/3")
HI_S4 = Namespace("https://w3id.org/hi-ontology/scenarios/4")
HI_S5 = Namespace("https://w3id.org/hi-ontology/scenarios/5")
HI_S6 = Namespace("https://w3id.org/hi-ontology/scenarios/6")
HI_S7 = Namespace("https://w3id.org/hi-ontology/scenarios/7")

g.bind("hi", HI)
g.bind("dul", DUL)
g.bind("hint", HINT)
g.bind("hi-s", HI_S1)
g.bind("hi-s", HI_S2)
g.bind("hi-s", HI_S3)
g.bind("hi-s", HI_S4)
g.bind("hi-s", HI_S5)
g.bind("hi-s", HI_S6)
g.bind("hi-s", HI_S7)

#### 🤖 CQ 1 🤖

**Given a use case, how is the hybrid intelligence team composed, and what roles do its agents assume?**

In [93]:
cq1 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX hi-s1: <https://w3id.org/hi-ontology/scenarios/1/>

SELECT DISTINCT ?agentType ?agentTypeLabel ?roleConcept ?roleLabel ?roleDefinition
WHERE {
  VALUES ?useCase { hi-s1:UseCasePersonalAssistantHI }
  ?useCase hi:hasHITeam ?team .
  ?team hi:hasMember ?member .
  ?member a ?agentType ;
      hi:hasRoleConcept ?roleConcept .
  ?agentType rdfs:label ?agentTypeLabel .
  ?roleConcept skos:prefLabel ?roleLabel ;
      skos:definition ?roleDefinition .
}
"""

results_cq1 = g.query(cq1)
df_cq1 = pd.DataFrame([(str(row.agentType), str(row.agentTypeLabel), str(row.roleConcept), str(row.roleLabel), str(row.roleDefinition)) for row in results_cq1],
                        columns=["Agent", "Agent Label", "Role Concept", "Role Label", "Role Definition"])
df_cq1

,Agent,Agent Label,Role Concept,Role Label,Role Definition
0,https://w3id.org/hi-ontology#ArtificialAgent,Artificial Agent,https://w3id.org/hi-thesaurus/AIAssistantRole,System Assistant Role,A role where the agent assists the system in p...
1,https://w3id.org/hi-ontology#HumanAgent,Human Agent,https://w3id.org/hi-thesaurus/HumanUserRole,User Role,A role where the agent represents the user int...


#### 🤖 CQ 2 🤖

**Given an agent's capabilities, which tasks can it perform, and in which use cases can it participate?**

In [94]:
cq2 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?task ?taskLabel ?taskDefinition ?useCase ?useCaseConcept ?useCaseLabel ?useCaseDefinition

WHERE {
  VALUES ?availableCapConcept { hint:KnowledgeGraphQueryingCapability hint:ExplainingCapability hint:SituationalAwarenessCapability }

  ?capability hi:hasCapabilityConcept ?availableCapConcept ;
      hi:allowsTask ?task .
  ?task hi:hasTaskConcept ?taskConcept ;
      hi:requiresCapability ?reqCap .
  ?taskConcept skos:prefLabel  ?taskLabel ;
      skos:definition ?taskDefinition .
  ?reqCap hi:hasCapabilityConcept ?reqCapConcept .

  FILTER NOT EXISTS {
    ?task hi:requiresCapability ?missingCap .
    ?missingCap hi:hasCapabilityConcept ?missingCapConcept .
    FILTER ( ?missingCapConcept NOT IN ( hint:KnowledgeGraphQueryingCapability, hint:ExplainingCapability, hint:SituationalAwarenessCapability ))
  }

  ?agent hi:hasCapability ?capability .
  ?useCase hi:hasHITeam ?team ;
      hi:hasUseCaseConcept ?useCaseConcept .
  ?team hi:hasMember ?agent .
  ?useCaseConcept skos:prefLabel  ?useCaseLabel ;
      skos:definition ?useCaseDefinition .
}
"""

results_cq2 = g.query(cq2)
df_cq2 = pd.DataFrame([(str(row.task), str(row.taskLabel), str(row.taskDefinition), str(row.useCase), 
                        str(row.useCaseConcept), str(row.useCaseLabel), str(row.useCaseDefinition)) for row in results_cq2],
                        columns=["Task", "Task Label", "Task Definition", "Use Case", "Use Case Concept", "Use Case Label", "Use Case Definition"])
df_cq2

,Task,Task Label,Task Definition,Use Case,Use Case Concept,Use Case Label,Use Case Definition
0,https://w3id.org/hi-ontology/scenarios/2/TaskK...,Knowledge Query Task,A task that involves querying knowledge graphs...,https://w3id.org/hi-ontology/scenarios/2/UseCa...,https://w3id.org/hi-thesaurus/CollaborativeMan...,Collaborative Manipulation Use Case,A use case where Hybrid Intelligence systems c...
1,https://w3id.org/hi-ontology/scenarios/4/TaskK...,Knowledge Query Task,A task that involves querying knowledge graphs...,https://w3id.org/hi-ontology/scenarios/4/UseCa...,https://w3id.org/hi-thesaurus/AutonomousTutori...,Autonomous Tutoring Use Case,A use case where Hybrid Intelligence systems p...
2,https://w3id.org/hi-ontology/scenarios/6/TaskL...,Learn Energy Behavior Task,A task that involves learning energy behavior.,https://w3id.org/hi-ontology/scenarios/6/UseCa...,https://w3id.org/hi-thesaurus/EnergyNegotiatio...,Energy Negotiation Use Case,A use case where Hybrid Intelligence systems e...
3,https://w3id.org/hi-ontology/scenarios/1/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/1/UseCa...,https://w3id.org/hi-thesaurus/PersonalAssistan...,Personal Assistant Use Case,A use case where Hybrid Intelligence systems f...
4,https://w3id.org/hi-ontology/scenarios/2/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/2/UseCa...,https://w3id.org/hi-thesaurus/CollaborativeMan...,Collaborative Manipulation Use Case,A use case where Hybrid Intelligence systems c...
5,https://w3id.org/hi-ontology/scenarios/3/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/3/UseCa...,https://w3id.org/hi-thesaurus/GroupDeliberatio...,Group Deliberation Use Case,A use case where Hybrid Intelligence systems f...
6,https://w3id.org/hi-ontology/scenarios/4/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/4/UseCa...,https://w3id.org/hi-thesaurus/AutonomousTutori...,Autonomous Tutoring Use Case,A use case where Hybrid Intelligence systems p...
7,https://w3id.org/hi-ontology/scenarios/5/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/5/UseCa...,https://w3id.org/hi-thesaurus/MedicalDiagnosis...,Medical Diagnosis Use Case,A use case where Hybrid Intelligence systems a...
8,https://w3id.org/hi-ontology/scenarios/6/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/6/UseCa...,https://w3id.org/hi-thesaurus/EnergyNegotiatio...,Energy Negotiation Use Case,A use case where Hybrid Intelligence systems e...
9,https://w3id.org/hi-ontology/scenarios/7/TaskE...,Explanation Task,A task that involves providing explanations or...,https://w3id.org/hi-ontology/scenarios/7/UseCa...,https://w3id.org/hi-thesaurus/VirtualMuseumGui...,Virtual Museum Guide Use Case,A use case where Hybrid Intelligence systems s...


#### 🤖 CQ 3 🤖

**What candidate team compositions satisfy all capability requirements for achieving a specific goal?**

In [95]:
cq3 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?team ?agent ?agentCapConcept ?agentCapLabel
WHERE {

  VALUES ?goalConcept { hint:FairAccurateVerdictGoal }

  ?goal hi:hasGoalConcept ?goalConcept ;
      hi:requiresTask ?task .
  ?task hi:requiresCapability ?reqCap .
  ?reqCap hi:hasCapabilityConcept ?reqCapConcept .
  ?goalConcept skos:prefLabel ?goalConceptLabel . 
  ?reqCapConcept skos:prefLabel ?reqCapLabel . 
  ?team a hi:HITeam .
  
  FILTER NOT EXISTS {
    ?goal hi:requiresTask ?t2 .
    ?t2 hi:requiresCapability ?cap2 .
    ?cap2 hi:hasCapabilityConcept ?reqCapConcept2 .
    FILTER NOT EXISTS {
      ?team hi:hasMember ?a2 .
      ?a2 hi:hasCapability ?a2Cap .
      ?a2Cap hi:hasCapabilityConcept ?reqCapConcept2 .
    }
  }

  ?team hi:hasMember ?agent .
  ?agent hi:hasCapability ?agentCap .
  ?agentCap hi:hasCapabilityConcept ?agentCapConcept .
  ?agentCapConcept skos:prefLabel ?agentCapLabel . 
}
"""

results_cq3 = g.query(cq3)
df_cq3 = pd.DataFrame([(str(row.team), str(row.agent), str(row.agentCapConcept), str(row.agentCapLabel)) for row in results_cq3],
                        columns=["Team", "Agent", "Agent Capability Concept", "Agent Capability Label"])
df_cq3

,Team,Agent,Agent Capability Concept,Agent Capability Label
0,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/AIDel...,https://w3id.org/hi-thesaurus/BayesianReasonin...,Bayesian reasoning capability
1,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/AIDel...,https://w3id.org/hi-thesaurus/MetaAnalysisCapa...,Meta-analysis capability
2,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/AIDel...,https://w3id.org/hi-thesaurus/ExplainingCapabi...,Explaining capability
3,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/Human...,https://w3id.org/hi-thesaurus/SelfReportingCap...,Self-reporting capability
4,https://w3id.org/hi-ontology/scenarios/3/HITea...,https://w3id.org/hi-ontology/scenarios/3/Human...,https://w3id.org/hi-thesaurus/SelfReportingCap...,Self-reporting capability
5,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/BayesianReasonin...,Bayesian reasoning capability
6,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/SemanticReasonin...,Semantic reasoning capability
7,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/KnowledgeGraphQu...,Knowledge graph querying capability
8,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/ExplainingCapabi...,Explaining capability
9,https://w3id.org/hi-ontology/scenarios/4/HITea...,https://w3id.org/hi-ontology/scenarios/4/Human...,https://w3id.org/hi-thesaurus/SelfReportingCap...,Self-reporting capability


#### 🤖 CQ 4 🤖

**How can we infer team eligibility, task assignment, or contextual influences?**

In [96]:
# swrl rules

#### 🤖 CQ 5 🤖

**What types of interactions occur between agents during the execution of a specific task?**

In [97]:
cq5 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?goalConcept ?task ?taskComment ?interaction ?agent 
                ?intentConcept ?intentLabel ?intentDefinition 
                ?modalityConcept ?modalityLabel ?modalityDefinition
WHERE {
    ?interaction a hi:Interaction ;
        hi:hasAgentInvolved ?agent ;
        hi:hasInteractionIntentConcept ?intentConcept ;
        hi:hasInteractionModalityConcept ?modalityConcept .  
    ?intentConcept skos:prefLabel ?intentLabel ;
        skos:definition ?intentDefinition .
    ?modalityConcept skos:prefLabel ?modalityLabel ;
        skos:definition ?modalityDefinition .          
    ?taskExecution hi:hasInteractionEpisode ?interaction ;
        hi:realizesTask ?task .
    ?task rdfs:comment ?taskComment .
    ?goal hi:requiresTask ?task ;
        hi:hasGoalConcept ?goalConcept .
}
"""

results_cq5 = g.query(cq5)
df_cq5 = pd.DataFrame([(str(row.goalConcept), str(row.task), str(row.taskComment), str(row.interaction), str(row.agent),
                        str(row.intentConcept), str(row.intentLabel), str(row.intentDefinition),
                        str(row.modalityConcept), str(row.modalityLabel), str(row.modalityDefinition)) 
                        for row in results_cq5],
                        columns=["Goal Concept", "Task", "Task Comment", "Interaction", "Agent",
                                    "Intent Concept", "Intent Label", "Intent Definition",
                                    "Modality Concept", "Modality Label", "Modality Definition"])
df_cq5

,Goal Concept,Task,Task Comment,Interaction,Agent,Intent Concept,Intent Label,Intent Definition,Modality Concept,Modality Label,Modality Definition
0,https://w3id.org/hi-thesaurus/UserWellbeingGoal,https://w3id.org/hi-ontology/scenarios/1/TaskE...,Provide explanations to maintain trust and eng...,https://w3id.org/hi-ontology/scenarios/1/Inter...,https://w3id.org/hi-ontology/scenarios/1/AIAss...,https://w3id.org/hi-thesaurus/ExplanationIntent,Explanation Intent,An intent focused on providing explanations or...,https://w3id.org/hi-thesaurus/VerbalDialogueMo...,Verbal Dialogue Modality,An interaction modality that involves verbal c...
1,https://w3id.org/hi-thesaurus/UserWellbeingGoal,https://w3id.org/hi-ontology/scenarios/1/TaskE...,Provide explanations to maintain trust and eng...,https://w3id.org/hi-ontology/scenarios/1/Inter...,https://w3id.org/hi-ontology/scenarios/1/Human...,https://w3id.org/hi-thesaurus/ExplanationIntent,Explanation Intent,An intent focused on providing explanations or...,https://w3id.org/hi-thesaurus/VerbalDialogueMo...,Verbal Dialogue Modality,An interaction modality that involves verbal c...
2,https://w3id.org/hi-thesaurus/EffectiveLearnin...,https://w3id.org/hi-ontology/scenarios/4/TaskE...,Explain why specific resources and topics were...,https://w3id.org/hi-ontology/scenarios/4/Inter...,https://w3id.org/hi-ontology/scenarios/4/AITutor,https://w3id.org/hi-thesaurus/ExplanationIntent,Explanation Intent,An intent focused on providing explanations or...,https://w3id.org/hi-thesaurus/TextChatModality,Text Chat Modality,An interaction modality that involves a text c...
3,https://w3id.org/hi-thesaurus/EffectiveLearnin...,https://w3id.org/hi-ontology/scenarios/4/TaskE...,Explain why specific resources and topics were...,https://w3id.org/hi-ontology/scenarios/4/Inter...,https://w3id.org/hi-ontology/scenarios/4/Human...,https://w3id.org/hi-thesaurus/ExplanationIntent,Explanation Intent,An intent focused on providing explanations or...,https://w3id.org/hi-thesaurus/TextChatModality,Text Chat Modality,An interaction modality that involves a text c...


#### 🤖 CQ 6 🤖

**Given a method, which goals and tasks can be achieved with it?**

In [98]:
cq6 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?goalLabel ?goalDefinition ?taskLabel ?taskDefinition
WHERE {
    VALUES ?method { hint:SemanticLinkingMethod }

    ?taskExecution hi:hasMethodConcept ?method ;
        hi:realizesTask ?task ;
        hi:towardsGoal ?goal .
    ?goal hi:hasGoalConcept ?goalConcept .
    ?goalConcept skos:prefLabel ?goalLabel ;
        skos:definition ?goalDefinition .
    ?task hi:hasTaskConcept ?taskConcept .
    ?taskConcept skos:prefLabel ?taskLabel ;
        skos:definition ?taskDefinition .
}
"""

results_cq6 = g.query(cq6)
df_cq6 = pd.DataFrame([(str(row.goalLabel), str(row.goalDefinition), str(row.taskLabel), str(row.taskDefinition)) for row in results_cq6],
                        columns=["Goal Label ", "Goal Definition", "Task Label", "Task Definition"])
df_cq6

,Goal Label,Goal Definition,Task Label,Task Definition
0,Effective Learning Goal,A goal focused on promoting effective learning...,Semantic Reasoning Task,A task that involves performing reasoning base...
1,Effective Learning Goal,A goal focused on promoting effective learning...,Recommend Resources Task,A task that involves recommending resources or...
2,Accurate and Trustworthy Diagnosis,A goal aimed at ensuring accurate and trustwor...,Candidate Hypothesis Generation Task,A task that involves generating candidate hypo...
3,Enhanced Visitor Goal,A goal aimed at enhancing visitors.,Recommend Resources Task,A task that involves recommending resources or...
4,Enhanced Visitor Goal,A goal aimed at enhancing visitors.,Explanation Task,A task that involves providing explanations or...


#### 🤖 CQ 7 🤖

**Given an evaluation metric, which task can be assessed with it?**

In [99]:
cq7 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?task ?taskConcept ?taskLabel ?taskDefinition
WHERE {
    VALUES ?metric { hint:FairOutcomeMetric }

    ?experiment hi:hasMetricTested ?metric .
    ?evaluation hi:hasExperiment ?experiment .
    ?execution hi:evaluatedBy ?evaluation ;
        hi:realizesTask ?task .
    ?task hi:hasTaskConcept ?taskConcept .
    ?taskConcept skos:prefLabel ?taskLabel ;
        skos:definition ?taskDefinition .
}
"""

results_cq7 = g.query(cq7)
df_cq7 = pd.DataFrame([(str(row.task), str(row.taskConcept), str(row.taskLabel), str(row.taskDefinition)) for row in results_cq7],
                        columns=["Task", "Task Concept", "Task Label", "Task Definition"])
df_cq7

,Task,Task Concept,Task Label,Task Definition
0,https://w3id.org/hi-ontology/scenarios/1/TaskC...,https://w3id.org/hi-thesaurus/ConflictResoluti...,Conflict Resolution Task,A task focused on resolving conflicts or disag...
1,https://w3id.org/hi-ontology/scenarios/3/TaskE...,https://w3id.org/hi-thesaurus/ExplanationTask,Explanation Task,A task that involves providing explanations or...
2,https://w3id.org/hi-ontology/scenarios/3/TaskD...,https://w3id.org/hi-thesaurus/DecideInterventi...,Decide Intervention Task,A task that involves deciding on appropriate i...
3,https://w3id.org/hi-ontology/scenarios/3/TaskP...,https://w3id.org/hi-thesaurus/ProvideStatement...,Provide Statement Task,A task that involves providing statements or r...
4,https://w3id.org/hi-ontology/scenarios/6/TaskE...,https://w3id.org/hi-thesaurus/ExplanationTask,Explanation Task,A task that involves providing explanations or...
5,https://w3id.org/hi-ontology/scenarios/6/TaskL...,https://w3id.org/hi-thesaurus/LearnOccupancyTask,Learn Occupancy Task,A task that involves learning occupancy.
6,https://w3id.org/hi-ontology/scenarios/6/TaskL...,https://w3id.org/hi-thesaurus/LearnEnergyBehav...,Learn Energy Behavior Task,A task that involves learning energy behavior.
7,https://w3id.org/hi-ontology/scenarios/6/TaskN...,https://w3id.org/hi-thesaurus/AllocationNegota...,Allocation Negotiation Task,A task that involves negotiation of energy all...
8,https://w3id.org/hi-ontology/scenarios/6/TaskC...,https://w3id.org/hi-thesaurus/OperatorCoordina...,Operator Coordination Task,A task that involves coordination with operator.


#### 🤖 CQ 8 🤖

**Given a goal, which experiments test which metrics, and what hypotheses do these experiments address?**

In [100]:
cq8 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?experiment ?experimentConcept ?experimentLabel ?experimentDefinition
                ?metricConcept ?metricLabel ?metricDefinition
                ?nullHypothesis ?alternativeHypothesis
WHERE {
    VALUES ?goalConcept { hint:EnhancedVisitorGoal }

    ?goal hi:hasGoalConcept ?goalConcept ;
        hi:requiresTask ?task .
    ?taskExecution hi:realizesTask ?task ;
        hi:evaluatedBy ?evaluation .
    ?evaluation hi:hasMetricConcept ?metricConcept ;
        hi:hasExperiment ?experiment .
    ?experiment hi:hasNullHypothesis ?nullHypothesis ;
        hi:hasAlternativeHypothesis ?alternativeHypothesis ;
        hi:hasExperimentConcept ?experimentConcept .
    ?experimentConcept skos:prefLabel ?experimentLabel ;
        skos:definition ?experimentDefinition .
    ?metricConcept skos:prefLabel ?metricLabel ;
        skos:definition ?metricDefinition .
}
"""

results_cq8 = g.query(cq8)
df_cq8 = pd.DataFrame([(str(row.experiment), str(row.experimentConcept), str(row.experimentLabel), str(row.experimentDefinition),
                        str(row.metricConcept), str(row.metricLabel), str(row.metricDefinition),
                        str(row.nullHypothesis), str(row.alternativeHypothesis)) for row in results_cq8],
                        columns=["Experiment", "Experiment Concept", "Experiment Label", "Experiment Definition",
                                "Metric Concept", "Metric Label", "Metric Definition",
                                "Null Hypothesis", "Alternative Hypothesis"])
df_cq8

,Experiment,Experiment Concept,Experiment Label,Experiment Definition,Metric Concept,Metric Label,Metric Definition,Null Hypothesis,Alternative Hypothesis
0,https://w3id.org/hi-ontology/scenarios/7/Exper...,https://w3id.org/hi-thesaurus/VirtualRealityEf...,Virtual Reality Effectiveness Experiment,An experiment designed to measure the effectiv...,https://w3id.org/hi-thesaurus/RecommendationRe...,Recommendation Relevance Metric,A metric that quantifies the relevance of reco...,No improvement over a non-personalized baseline.,Multimodal input + KG reasoning improves engag...
1,https://w3id.org/hi-ontology/scenarios/7/Exper...,https://w3id.org/hi-thesaurus/VirtualRealityEf...,Virtual Reality Effectiveness Experiment,An experiment designed to measure the effectiv...,https://w3id.org/hi-thesaurus/AnswerAccuracyMe...,Answer Accuracy Metric,A metric that quantifies the accuracy of an an...,No improvement over a non-personalized baseline.,Multimodal input + KG reasoning improves engag...
2,https://w3id.org/hi-ontology/scenarios/7/Exper...,https://w3id.org/hi-thesaurus/VirtualRealityEf...,Virtual Reality Effectiveness Experiment,An experiment designed to measure the effectiv...,https://w3id.org/hi-thesaurus/EngagementTimeMe...,Engagement Time Metric,A metric that quantifies the engagement time.,No improvement over a non-personalized baseline.,Multimodal input + KG reasoning improves engag...
3,https://w3id.org/hi-ontology/scenarios/7/Exper...,https://w3id.org/hi-thesaurus/VirtualRealityEf...,Virtual Reality Effectiveness Experiment,An experiment designed to measure the effectiv...,https://w3id.org/hi-thesaurus/ExplanationClari...,Explanation Clarity Metric,A metric that assesses the clarity and compreh...,No improvement over a non-personalized baseline.,Multimodal input + KG reasoning improves engag...


#### 🤖 CQ 9 🤖

**Given a contextual constraint, which use cases are associated with it, and how does it affect the team?**

In [101]:
cq9 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?useCase ?useCaseConcept ?useCaseLabel ?useCaseDefinition 
                ?team ?member ?roleConcept ?roleLabel ?roleDefinition
WHERE {
    VALUES ?constraint { hint:PrivacyConstraint }

    ?context hi:hasConstraintConcept ?constraint ;
        hi:hasInfluenceOn ?team .
    ?useCase hi:hasHITeam ?team ;
        hi:hasUseCaseConcept ?useCaseConcept .
    ?useCaseConcept skos:prefLabel ?useCaseLabel ;
        skos:definition ?useCaseDefinition .
    ?team hi:hasMember ?member .
    ?member hi:hasRoleConcept ?roleConcept .
    ?roleConcept skos:prefLabel ?roleLabel ;
        skos:definition ?roleDefinition
}
"""

results_cq9 = g.query(cq9)
df_cq9 = pd.DataFrame([(str(row.useCase), str(row.useCaseConcept), str(row.useCaseLabel), str(row.useCaseDefinition),
                        str(row.team), str(row.member), str(row.roleConcept), str(row.roleLabel), str(row.roleDefinition)) for row in results_cq9],
                        columns=["Use Case", "Use Case Concept", "Use Case Label", "Use Case Definition",
                                "Team", "Member", "Role Concept", "Role Label", "Role Definition"])
df_cq9

,Use Case,Use Case Concept,Use Case Label,Use Case Definition,Team,Member,Role Concept,Role Label,Role Definition
0,https://w3id.org/hi-ontology/scenarios/1/UseCa...,https://w3id.org/hi-thesaurus/PersonalAssistan...,Personal Assistant Use Case,A use case where Hybrid Intelligence systems f...,https://w3id.org/hi-ontology/scenarios/1/HITea...,https://w3id.org/hi-ontology/scenarios/1/AIAss...,https://w3id.org/hi-thesaurus/AIAssistantRole,System Assistant Role,A role where the agent assists the system in p...
1,https://w3id.org/hi-ontology/scenarios/1/UseCa...,https://w3id.org/hi-thesaurus/PersonalAssistan...,Personal Assistant Use Case,A use case where Hybrid Intelligence systems f...,https://w3id.org/hi-ontology/scenarios/1/HITea...,https://w3id.org/hi-ontology/scenarios/1/Human...,https://w3id.org/hi-thesaurus/HumanUserRole,User Role,A role where the agent represents the user int...
2,https://w3id.org/hi-ontology/scenarios/7/UseCa...,https://w3id.org/hi-thesaurus/VirtualMuseumGui...,Virtual Museum Guide Use Case,A use case where Hybrid Intelligence systems s...,https://w3id.org/hi-ontology/scenarios/7/HITea...,https://w3id.org/hi-ontology/scenarios/7/AIVir...,https://w3id.org/hi-thesaurus/VirtualGuideRole,Virtual Guide Role,A role where the agent represents the virtual ...
3,https://w3id.org/hi-ontology/scenarios/7/UseCa...,https://w3id.org/hi-thesaurus/VirtualMuseumGui...,Virtual Museum Guide Use Case,A use case where Hybrid Intelligence systems s...,https://w3id.org/hi-ontology/scenarios/7/HITea...,https://w3id.org/hi-ontology/scenarios/7/Human...,https://w3id.org/hi-thesaurus/VisitorRole,Visitor Role,A role where the agent represents the visitor.


#### 🤖 CQ 10 🤖

**Which constraints are shared across different use cases and agents?**

In [102]:
cq10 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>

SELECT DISTINCT ?useCaseConcept ?useCaseLabel ?useCaseDefinition 
                ?member ?roleConcept ?roleLabel ?roleDefinition
WHERE {
    ?context hi:hasConstraintConcept ?constraintConcept ;
        hi:hasInfluenceOn ?team .
    ?team hi:hasMember ?member .
    ?member hi:hasRoleConcept ?roleConcept .
    ?roleConcept skos:prefLabel ?roleLabel ;
        skos:definition ?roleDefinition .
    ?useCase hi:hasHITeam ?team ;
        hi:hasUseCaseConcept ?useCaseConcept .
    ?useCaseConcept skos:prefLabel ?useCaseLabel ;
        skos:definition ?useCaseDefinition .
}
"""

results_cq10 = g.query(cq10)
df_cq10 = pd.DataFrame([(str(row.useCaseConcept), str(row.useCaseLabel), str(row.useCaseDefinition),
                        str(row.member), str(row.roleConcept), str(row.roleLabel), str(row.roleDefinition)) for row in results_cq10],
                        columns=["Use Case Concept", "Use Case Label", "Use Case Definition",
                                    "Member", "Role Concept", "Role Label", "Role Definition"])
df_cq10

,Use Case Concept,Use Case Label,Use Case Definition,Member,Role Concept,Role Label,Role Definition
0,https://w3id.org/hi-thesaurus/PersonalAssistan...,Personal Assistant Use Case,A use case where Hybrid Intelligence systems f...,https://w3id.org/hi-ontology/scenarios/1/AIAss...,https://w3id.org/hi-thesaurus/AIAssistantRole,System Assistant Role,A role where the agent assists the system in p...
1,https://w3id.org/hi-thesaurus/PersonalAssistan...,Personal Assistant Use Case,A use case where Hybrid Intelligence systems f...,https://w3id.org/hi-ontology/scenarios/1/Human...,https://w3id.org/hi-thesaurus/HumanUserRole,User Role,A role where the agent represents the user int...
2,https://w3id.org/hi-thesaurus/CollaborativeMan...,Collaborative Manipulation Use Case,A use case where Hybrid Intelligence systems c...,https://w3id.org/hi-ontology/scenarios/2/Robot...,https://w3id.org/hi-thesaurus/RiskOperatorRole,Risk Operator Role,A role where the agent is responsible for mana...
3,https://w3id.org/hi-thesaurus/CollaborativeMan...,Collaborative Manipulation Use Case,A use case where Hybrid Intelligence systems c...,https://w3id.org/hi-ontology/scenarios/2/Human...,https://w3id.org/hi-thesaurus/FineManipulatorRole,Fine Manipulator Role,A role where the agent performs precise manipu...
4,https://w3id.org/hi-thesaurus/GroupDeliberatio...,Group Deliberation Use Case,A use case where Hybrid Intelligence systems f...,https://w3id.org/hi-ontology/scenarios/3/AIDel...,https://w3id.org/hi-thesaurus/AIAdvisorRole,AI Advisor Role,A role where the agent provides advice and rec...
5,https://w3id.org/hi-thesaurus/GroupDeliberatio...,Group Deliberation Use Case,A use case where Hybrid Intelligence systems f...,https://w3id.org/hi-ontology/scenarios/3/Human...,https://w3id.org/hi-thesaurus/JurorRole,Juror Role,A role where the agent participates in jury de...
6,https://w3id.org/hi-thesaurus/GroupDeliberatio...,Group Deliberation Use Case,A use case where Hybrid Intelligence systems f...,https://w3id.org/hi-ontology/scenarios/3/Human...,https://w3id.org/hi-thesaurus/JurorRole,Juror Role,A role where the agent participates in jury de...
7,https://w3id.org/hi-thesaurus/EnergyNegotiatio...,Energy Negotiation Use Case,A use case where Hybrid Intelligence systems e...,https://w3id.org/hi-ontology/scenarios/6/AIBui...,https://w3id.org/hi-thesaurus/BuilderRole,Builder Role,A role where the agent represents the builder.
8,https://w3id.org/hi-thesaurus/EnergyNegotiatio...,Energy Negotiation Use Case,A use case where Hybrid Intelligence systems e...,https://w3id.org/hi-ontology/scenarios/6/Human...,https://w3id.org/hi-thesaurus/GridOperatorRole,Grid Operator Role,A role where the agent represents the grid ope...
9,https://w3id.org/hi-thesaurus/VirtualMuseumGui...,Virtual Museum Guide Use Case,A use case where Hybrid Intelligence systems s...,https://w3id.org/hi-ontology/scenarios/7/AIVir...,https://w3id.org/hi-thesaurus/VirtualGuideRole,Virtual Guide Role,A role where the agent represents the virtual ...


#### 🤖 CQ 11 🤖

**Given a use case, which contextual phenomena are associated with it?**

In [103]:
cq11 = """
PREFIX hi:    <https://w3id.org/hi-ontology#>
PREFIX hint:  <https://w3id.org/hi-thesaurus/>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX hi-s6: <https://w3id.org/hi-ontology/scenarios/6/> 

SELECT DISTINCT ?phenomenonConcept ?phenomenonLabel ?phenomenonDefinition
WHERE {
    VALUES ?useCase { hi-s6:UseCaseCampusEnergyNegotiation }

    ?useCase hi:hasHITeam ?team .
    ?context hi:hasInfluenceOn ?team ;
        hi:hasPhenomenonConcept ?phenomenonConcept .
    ?phenomenonConcept skos:prefLabel ?phenomenonLabel ;
        skos:definition ?phenomenonDefinition .
}
"""

results_cq11 = g.query(cq11)
df_cq11 = pd.DataFrame([(str(row.phenomenonConcept), str(row.phenomenonLabel), str(row.phenomenonDefinition)) for row in results_cq11],
                        columns=["Phenomenon Concept", "Phenomenon Label", "Phenomenon Definition"])
df_cq11

,Phenomenon Concept,Phenomenon Label,Phenomenon Definition
0,https://w3id.org/hi-thesaurus/LoadVariabilityP...,Load Variability Phenomenon,"Fluctuations in workload, energy demand, or sy..."
1,https://w3id.org/hi-thesaurus/CooperativeLearn...,Cooperative Learning Dynamics Phenomenon,A phenomenon where agents iteratively improve ...


#### 🤖 CQ 12 🤖

**Under what conditions does an agent's behaviour comply with or violate a contextual constraint associated with a use case?**